# Spam Classifiers
I use this notebook in showcasing multiple algorithms for performing a binary classification task on the Spambase dataset. 

The dataset has the structure:
- 4601 Examples
- 57 features
- 1 Label:
    - 0 - notSpam - 2788 examples
    - 1 - spam - 1813 examples

We start off by importing the necessary packages. We need to be able to read and write CSV files (csv), perform matrix computations (numpy) and graph our results (matplotlib). TensorFlow provides a streamlined way to implement multiple learning algorithms quickly. 

In [6]:
import csv
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

Next, we set some global variables for the script. The filename, hyperparameters (step size, number of epochs, momentum, batch size), the feature dimension (57) and number of output classes (2).

In [108]:
# filename containing the dataset
filename = 'Datasets/Spambase/spambase.data'

# Hyperparameters
numEpochs = 15
stepSize = 0.95e-3
batchSize = 20
momentum = 0.785

# Information about the data
featureDimension = 57
numClasses = 2

# The percentage of data to use for training
trainRatio = 0.8

With these defined, we can then define some helper functions that manipulate the data.

In [30]:
# Load the data from the filename and return the spam and notSpam arrays
def loadData(filename):
    data = np.array(list(csv.reader(open(filename), delimiter=',', 
            quoting=csv.QUOTE_NONNUMERIC)))
    spam = data[:1813, :]
    notSpam = data[1813:, :]
    return spam, notSpam

# Shuffle, then plit the data according to the 
# train-test ratio (percent - 0.8)
def splitData(spam, notSpam, trainRatio, seed):
    # Shuffle the spam and notSpam
    np.random.seed(seed)
    np.random.shuffle(spam)
    np.random.shuffle(notSpam)
    
    # Split the data according to the ratio
    numSpamTrain = int(trainRatio*spam.shape[0] + 1)
    numNotTrain = int(trainRatio*notSpam.shape[0] + 1)
    
    spamTrain = spam[:numSpamTrain, :]
    spamTest = spam[numSpamTrain:, :]
    
    notTrain = notSpam[:numNotTrain, :]
    notTest = notSpam[numNotTrain:, :]
    
    # Return the arrays still separated by class
    return spamTrain, spamTest, notTrain, notTest

# Takes only a percentage of the training data and returns 
# the concatenated array
# For using only a subset of the training data
def takePercentData(spamTrain, notTrain, percentage, seed):
    percentage /= 100.
    
    numSpam = int(percentage*spamTrain.shape[0] + 1)
    numNot = int(percentage*notTrain.shape[0] + 1)
    
    trainData = spamTrain[:numSpam, :]
    trainData = np.append(trainData, notTrain[:numNot, :], axis=0)
    
    np.random.shuffle(trainData)
    
    return trainData

Main function

In [102]:
def main():
    # Load the data
    spamData, notData = loadData(filename)
    curSplit = 1
    spamTrain, spamTest, notTrain, notTest = splitData(spamData, notData, trainRatio, curSplit)
    
    # Create the test data
    XTest = np.append(spamTest, notTest, axis=0)
#     print(XTest.shape)
    np.random.shuffle(XTest)
    
    YTest = XTest[:, -1]
    XTest = XTest[:, :-1]
    
    # Take the desired percentage of train data
    percentage = 30
    XTrain = takePercentData(spamTrain, notTrain, percentage, curSplit)
    np.random.shuffle(XTrain)
    YTrain = XTrain[:, -1]
    XTrain = XTrain[:, :-1]
    
    # Create a tf model
     # Define the input layer
    input = (keras.Input(shape = (featureDimension,), name='input'))

    numHiddenNeurons = 10
    activation = 'relu'
    numLayers = 2
    optChoice = 'adam'
    
    # Define first hidden layer
    hidden1 = (keras.layers.Dense(numHiddenNeurons, 
        kernel_regularizer=keras.regularizers.l2(100),
        activation=activation, name='hidden')(input))
    
    # If specified multiple layers, create hidden2, else go to output
    if numLayers == 2:
        hidden2 = (keras.layers.Dense(numHiddenNeurons, 
                    kernel_regularizer=keras.regularizers.l2(100),
                    activation=activation, name='hidden2')(hidden1))
        output = (keras.layers.Dense(2, activation='softmax', name='output')(hidden2))
    else:        
        output = (keras.layers.Dense(2, activation='softmax', name='output')(hidden1))

    # Put the model together and return it
    model = keras.Model(inputs=input, outputs=output, name='NN')
    
    opt = tf.keras.optimizers.SGD(learning_rate=stepSize, 
        momentum=momentum) if (optChoice == 'sgd'
        ) else tf.keras.optimizers.Adam(learning_rate=stepSize)
    
    # Compile the model with the optimizer, target metrics, and loss
    model.compile(
        optimizer = opt,
        loss = keras.losses.SparseCategoricalCrossentropy(),
        metrics = ['accuracy']
    )

    # Save the model diagram
    # saveModelDiagram(model)
    
    history = model.fit(XTrain, YTrain, epochs=numEpochs, 
                    validation_data=(XTest, YTest), verbose=1,
                    callbacks=[keras.callbacks.EarlyStopping()])
    print(history.history["val_accuracy"][-1])

In [ ]:
if __name__ == "__main__":
    main()